In [4]:

from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,SnowballStemmer,WordNetLemmatizer
import json
import os
from tqdm.notebook import tqdm,tnrange
import string
import numpy as np
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import defaultdict
from tabulate import tabulate

nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /home/sandeep/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/sandeep/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
def stripSpecialChar(text):
    return ''.join(ch for ch in text if ch.isalnum() and not ch.isdigit() and ch not in string.punctuation)

def preProcess(text):
    # stemmer = SnowballStemmer("english")
    stopWords = set(stopwords.words('english'))

    text = text.lower()                                     # convert all text to lower case
    text_tokens = word_tokenize(text)                       # tokenizing the text

    # stemmedWords = list([stemmer.stem(word) for word in text_tokens])
    # validTokens = [i for i in stemmedWords if i not in stopWords]

    validTokens = [i for i in text_tokens if i not in stopWords]    # removing stop words

    validTokens = [stripSpecialChar(x) for x in validTokens]   # stripping special characters
    validTokens = [x for x in validTokens if len(x) > 1]    # Choosing only words which has length > 1
    return set(validTokens)

In [ ]:
class PositionalIndex():
    def __init__(self):
        pass



    